In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Plots
using ProgressMeter
import ForwardDiff as FD
import Convex as cvx
import ECOS

include(joinpath(@__DIR__,"utils","fmincon.jl"))

In [ ]:
params = (
        m = 1600 # kg
        ρ = 1020 # kg/m3
        g = 9.81 # m/s2
        GMₜ = 2.4 # m
        Ixx = 6.8e3 # kg m2
        Kₚ = -1.0e3 # kg m2
        ∇ = m/ρ # m3
        h2 = 0.95 # m
        As = 22.8  # m^2, sail area
        Ak = 0.93 # m^2, keel area
        l1 = 1  # m, distance from the center of mass to the sail
        l2 = 1  # m, distance from the center of mass to the rudder
        h = 1  # m, height of the sail
        )

The state of the sailboat is:
$$
\eta = \begin{bmatrix}
x \\
y \\
\phi \\
\psi
\end{bmatrix}

\nu = \begin{bmatrix}
u \\
v \\
p \\
r
\end{bmatrix}.
$$

where x and y are north and east positions, $phi$ and $psi$ are the roll and yaw angles.

Where u and v are the linear velocities, p and r are the angular velocity of roll and yaw.

Course angle is defined by 
$$ X = \phi + \beta $$

where $\phi$ is the yaw (heading angle), β is the drift angle


$$ \beta = \arctan(\frac{v \sin(\beta)}{v \cos(\beta) + v_{w}}) $$

In a heading controller we minimise the following:
$$ e_1 = \phi - \phi_d,  e_2 = r$$
Where $\phi_d$ is the desired heading angle, r is the angular yaw velocity

The heading controller is turned into a course controller by adding a small correction term:

$$ e_1 = \phi - \phi_d
       = \phi - (X_d - \beta_b)     $$

$ \beta_b$ is the neccesary drift angle needed to achieve the desired course angle $X_d$.



In [ ]:

function Linearised_lift_sail(params, α, V, CL0, A)
    """
    Linearised lift force FL0 on sail or rudder
    ρ is the density of wind or water
    V is the velocity of the wind or water relative to the foil 
    CL0 is the linearised lift coefficient of the foil
    α is the angle of attack of the foil
    """
    ρ, As = params.ρ, params.As
    return 0.5 * \rho * A * \alpha * V^2 * CL0
end 

function Linearised_lift_keel(params, α, V, CL0, A)
    """
    Linearised lift force FL0 on sail or rudder
    ρ is the density of wind or water
    V is the velocity of the wind or water relative to the foil 
    CL0 is the linearised lift coefficient of the foil
    α is the angle of attack of the foil
    """
    ρ, Ak = params.ρ, params.A 
    return 0.5 * \rho * Ak * \alpha * V^2 * CL0
end 

function undesirable_force_simple(Sy, Ry)
    """
    Simpler equation for the undesirable force.
    Taken from https://doi.org/10.1016/j.ifacol.2016.10.490 
    """
    # Sy = Force from sail in y' direction
    # Ry = Force from rudder in y' direction

    return -Sy - Ry
end

function keel_angle_of_attack(params, Fu, Cl0k, u, v, αk)
    """
    Cl0k is the linearised lift coefficient of the keel
    """

    ρ, Ak = params.ρ, params.Ak
    αₖ = 2 * Fu / (ρ * Ak * Cl0k * (u^2 + v^2))
    return αₖ
end

function neccessary_drift_angle(params, αₖ, ϕ)
    """
    Calculate the neccessary drift angle for the boat to sail in the desired direction.
    """
    return -αₖ * cos(ϕ)
end


    
